In [1]:
import pandas as pd
import numpy as np

In [2]:
plays = pd.read_csv('../data/user_artists.dat', sep='\t')
artists = pd.read_csv('../data/artists.dat', sep='\t', usecols=['id','name'])

# Merge artist and user pref data
ap = pd.merge(artists, plays, how="inner", left_on="id", right_on="artistID")
ap = ap.rename(columns={"weight": "playCount"})

# Group artist by name
artist_rank = ap.groupby(['name']) \
    .agg({'userID' : 'count', 'playCount' : 'sum'}) \
    .rename(columns={"userID" : 'totalUsers', "playCount" : "totalPlays"}) \
    .sort_values(['totalPlays'], ascending=False)

artist_rank['avgPlays'] = artist_rank['totalPlays'] / artist_rank['totalUsers']
print(artist_rank)

                    totalUsers  totalPlays     avgPlays
name                                                   
Britney Spears             522     2393140  4584.559387
Depeche Mode               282     1301308  4614.567376
Lady Gaga                  611     1291387  2113.563011
Christina Aguilera         407     1058405  2600.503686
Paramore                   399      963449  2414.659148
...                        ...         ...          ...
Morris                       1           1     1.000000
Eddie Kendricks              1           1     1.000000
Excess Pressure              1           1     1.000000
My Mine                      1           1     1.000000
A.M. Architect               1           1     1.000000

[17632 rows x 3 columns]


In [3]:
# Merge into ap matrix
ap = ap.join(artist_rank, on="name", how="inner") \
    .sort_values(['playCount'], ascending=False)

# Preprocessing
pc = ap.playCount
play_count_scaled = (pc - pc.min()) / (pc.max() - pc.min())
ap = ap.assign(playCountScaled=play_count_scaled)
#print(ap)

# Build a user-artist rating matrix 
ratings_df = ap.pivot(index='userID', columns='artistID', values='playCountScaled')
ratings = ratings_df.fillna(0).values

# Show sparsity
sparsity = float(len(ratings.nonzero()[0])) / (ratings.shape[0] * ratings.shape[1]) * 100
print("sparsity: %.2f" % sparsity)

sparsity: 0.28


In [4]:
from scipy.sparse import csr_matrix

# Build a sparse matrix
X = csr_matrix(ratings)

n_users, n_items = ratings_df.shape
print("rating matrix shape", ratings_df.shape)

user_ids = ratings_df.index.values
artist_names = ap.sort_values("artistID")["name"].unique()

rating matrix shape (1892, 17632)


In [5]:
from lightfm import LightFM
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
from lightfm.cross_validation import random_train_test_split
from lightfm.data import Dataset

# Build data references + train test
Xcoo = X.tocoo()
data = Dataset()
data.fit(np.arange(n_users), np.arange(n_items))
interactions, weights = data.build_interactions(zip(Xcoo.row, Xcoo.col, Xcoo.data)) 
train, test = random_train_test_split(interactions)

# Ignore that (weight seems to be ignored...)
#train = train_.tocsr()
#test = test_.tocsr()
#train[train==1] = X[train==1]
#test[test==1] = X[test==1]

# To be completed...

In [6]:
# Train
model = LightFM(learning_rate=0.05, loss='warp')
model.fit(train, epochs=10, num_threads=2)

In [7]:
# Evaluate
train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))
res_warp = {'name':'warp','train_precision':train_precision,'test_precision':test_precision,'train_auc':test_auc,'test_auc':test_auc}


Precision: train 0.38, test 0.13.
AUC: train 0.97, test 0.86.


In [8]:
# Predict
scores = model.predict(0, np.arange(n_items))
top_items = artist_names[np.argsort(-scores)]
print(top_items)

['Depeche Mode' 'Muse' 'Coldplay' ... 'White Zombie' 'Pretty Boy Floyd'
 'Warlock']


In [9]:
model = LightFM(learning_rate=0.05, loss='bpr')
model.fit(train, epochs=10, num_threads=2)

In [10]:
# Evaluate
train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))
res_bpr = {'name':'bpr','train_precision':train_precision,'test_precision':test_precision,'train_auc':test_auc,'test_auc':test_auc}

Precision: train 0.36, test 0.11.
AUC: train 0.84, test 0.77.


In [11]:
# Predict
scores = model.predict(0, np.arange(n_items))
top_items = artist_names[np.argsort(-scores)]
print(top_items)

['The Beatles' 'Radiohead' 'Coldplay' ... 'Rihanna' 'Britney Spears'
 'Christina Aguilera']


In [12]:
model = LightFM(learning_rate=0.05, loss='logistic')
model.fit(train, epochs=10, num_threads=2)

In [13]:
# Evaluate
train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))
res_logistic = {'name':'logistic','train_precision':train_precision,'test_precision':test_precision,'train_auc':test_auc,'test_auc':test_auc}

Precision: train 0.20, test 0.07.
AUC: train 0.89, test 0.81.


In [14]:
# Predict
scores = model.predict(0, np.arange(n_items))
top_items = artist_names[np.argsort(-scores)]
print(top_items)

['Lady Gaga' 'Britney Spears' 'Rihanna' ... 'Taruna' 'Jonna Lee'
 'Forbidden Society & Current Value']


In [15]:
model = LightFM(learning_rate=0.05, loss='warp-kos')
model.fit(train, epochs=10, num_threads=2)

In [16]:
# Evaluate
train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))
res_warp_kos = {'name':'warp_kos','train_precision':train_precision,'test_precision':test_precision,'train_auc':test_auc,'test_auc':test_auc}

Precision: train 0.34, test 0.12.
AUC: train 0.89, test 0.82.


In [17]:
# Predict
scores = model.predict(0, np.arange(n_items))
top_items = artist_names[np.argsort(-scores)]
print(top_items)

['The Beatles' 'Coldplay' 'Nirvana' ... 'Irena Jarocka' 'Lifelover'
 'Greg Porée']


In [18]:
print("res_warp : ",res_warp)
print("res_bpr : ",res_bpr)
print("res_warp_kos : ",res_warp_kos)
data = [res_warp,res_bpr,res_warp_kos]

res_dataframe = pd.DataFrame(data = data , columns=['name','train_precision','test_precision','train_auc','test_auc'])
res_dataframe

res_warp :  {'name': 'warp', 'train_precision': 0.38270557, 'test_precision': 0.12555912, 'train_auc': 0.85873085, 'test_auc': 0.85873085}
res_bpr :  {'name': 'bpr', 'train_precision': 0.35506633, 'test_precision': 0.11235358, 'train_auc': 0.77378017, 'test_auc': 0.77378017}
res_warp_kos :  {'name': 'warp_kos', 'train_precision': 0.34228116, 'test_precision': 0.116347186, 'train_auc': 0.81940156, 'test_auc': 0.81940156}


,name,train_precision,test_precision,train_auc,test_auc
0,warp,0.382706,0.125559,0.858731,0.858731
1,bpr,0.355066,0.112354,0.773780,0.773780
2,warp_kos,0.342281,0.116347,0.819402,0.819402


In [19]:
df = pd.DataFrame(columns=["WRAP", "LOGISTIC", "BRP" ,"KOS-WRAP" ])

loss = ['warp', 'logistic', 'bpr', 'warp-kos' ]

for i,j in enumerate(loss):

    model = LightFM(learning_rate=0.05, loss= j )
    model.fit(train, epochs=10, num_threads=2)
    a = precision_at_k(model, train, k=5).mean()
    b = precision_at_k(model, test, k=5, train_interactions=train).mean()
    c = auc_score(model, train).mean()
    d = auc_score(model, test, train_interactions=train).mean()

    this_column = df.columns[i]
    df[this_column] = [a,b,c,d]

print(df)

       WRAP  LOGISTIC       BRP  KOS-WRAP
0  0.423873  0.218462  0.441592  0.387480
1  0.165495  0.086155  0.162833  0.153781
2  0.965216  0.887490  0.850794  0.887652
3  0.858185  0.808747  0.780025  0.822070


In [ ]:
def get_recommendations_svd(title,n_films=10):
    # Get the index of the movie that matches the title
    idx = indices[title]
    
    cosine_sim = linear_kernel(tfidf_matrix_svd, tfidf_matrix_svd[idx].reshape(1,-1))
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim))
    
    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:n_films + 1]
    
    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]
    
    # Return the top 10 most similar movies
    return metadata['title'].iloc[movie_indices]